In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/Users/tj/dcu/Group-Project


# Data

In [55]:
from src.features.basic import features
from src.features.common import postproc_default
from src.dataset.tc4tl import make_datasets
from src.hyperopt import RFP_PARAMS

In [56]:
trainset, testset, devset = make_datasets(features, postproc_default, tunables=RFP_PARAMS, verbose=True)

100%|██████████| 8423/8423 [00:29<00:00, 287.71it/s]


## Evaluate Basic Feature

In [57]:
from src.models.base import dual_evaluation
import pandas as pd

f1_report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=["RssiMin", "RssiMax", "RssiMean", "CoarseGrain"],
                    target="Distance")

In [58]:
print(f1_report['cg=0'])

              precision    recall  f1-score   support

         1.8       0.83      0.88      0.85       137
         4.5       0.89      0.85      0.87       159

    accuracy                           0.86       296
   macro avg       0.86      0.86      0.86       296
weighted avg       0.86      0.86      0.86       296



In [59]:
print(f1_report['cg=1'])

              precision    recall  f1-score   support

         1.2       0.79      0.77      0.78       153
         1.8       0.64      0.71      0.67       152
         3.0       0.73      0.67      0.70       175
         4.5       0.73      0.74      0.73       159

    accuracy                           0.72       639
   macro avg       0.72      0.72      0.72       639
weighted avg       0.72      0.72      0.72       639



In [60]:
from src.ndcf import compute_ndcf

compute_ndcf(sysout)

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.228758,0.063786,0.292544
1,fine_grain,1.8,0.144262,0.170659,0.314921
2,fine_grain,3.0,0.091667,0.264151,0.355818
3,coarse_grain,1.8,0.124088,0.150943,0.275031
